In [2]:
!pwd
!sh 'scripts/lgcn/train_lgcn_ms1m.sh'

/dcs/pg20/u2085214/fc/learn-to-cluster
Traceback (most recent call last):
  File "lgcn/main.py", line 4, in <module>
    import torch
ImportError: No module named torch
Traceback (most recent call last):
  File "lgcn/main.py", line 4, in <module>
    import torch
ImportError: No module named torch


In [41]:
import os
import torch

from mmcv import Config

import sys
from utils import (create_logger, set_random_seed, rm_suffix,
                   mkdir_if_no_exists)

from gae.models import build_model
from gae import build_handler

In [42]:
cfg_name = 'cfg_train_gae_cut.py'
config='./gae/configs/cfg_train_gae_cut.py' # written in script 
cfg = Config.fromfile(config)
cfg.cuda = torch.cuda.is_available()
print('cuda available:',cfg.cuda)

if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
if cfg.get('cudnn_deterministic', False):
    torch.backends.cudnn.deterministic = Trueprint(cfg.cuda)

setattr(cfg,'work_dir','./data/work_dir/'+cfg_name)
mkdir_if_no_exists(cfg.work_dir, is_folder=True)

cfg.phase = 'train' # written in script

cfg.load_from = None
cfg.resume_from = None

cfg.gpus = 1
cfg.distributed = False
cfg.save_output = False
cfg.no_cuda = False
cfg.force = False

cfg.seed = 1


cuda available: False


In [43]:
logger = create_logger()

if cfg.seed is not None:
    logger.info('Set random seed to {}'.format(cfg.seed))
    set_random_seed(cfg.seed)

[2021-07-06 05:17:00,993] Set random seed to 1
[2021-07-06 05:17:00,993] Set random seed to 1
[2021-07-06 05:17:00,993] Set random seed to 1
[2021-07-06 05:17:00,993] Set random seed to 1
2021-07-06 05:17:00,993 - INFO - Set random seed to 1


In [44]:
model = build_model(cfg.model['type'], **cfg.model['kwargs']) #build model: odel = dict(type='gae', kwargs=dict(feature_dim=256))
handler = build_handler(cfg.phase) #tain_gae.py or test_gae.py <= args.pahse:choices=['test', 'train']

# handler(model, cfg, logger) #e.g. call train_gae(gae,cfg,tracking event)

In [45]:

from collections import OrderedDict

import torch
from mmcv.runner import Runner, obj_from_dict
from mmcv.parallel import MMDataParallel
from gae.datasets import build_dataset, build_dataloader
from gae.online_evaluation import online_evaluate


In [46]:
for k, v in cfg.model['kwargs'].items(): #kwargs=dict(feature_dim=256)
    setattr(cfg.train_data, k, v) #k? v?
dataset = build_dataset(cfg.train_data)

data_loaders = [build_dataloader(dataset,
                        cfg.batch_size_per_gpu,
                        cfg.workers_per_gpu,
                        train=True,
                        shuffle=True)]

if cfg.distributed:
    raise NotImplementedError
#call _single_train

[./data/labels/part0_train_cut.meta] #cls: 4, #inst: 300
[Time] read meta and feature consumes 0.0043 s
[Time] read knn graph consumes 0.0095 s
feature shape: (300, 256), norm_feat: True, sort_knns: True k_at_hop: [40, 5], active_connection: 10
labels shape: (300,)
knns_graph shape: (300, 80)


In [47]:
#_single_train
if cfg.gpus > 1:
    raise NotImplemented

# model = MMDataParallel(model, device_ids=range(cfg.gpus))#.cuda()

In [48]:
def build_optimizer(model, optimizer_cfg):
    """Build optimizer from configs.
    """
    if hasattr(model, 'module'):
        model = model.module

    optimizer_cfg = optimizer_cfg.copy() #dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=1e-4)
    paramwise_options = optimizer_cfg.pop('paramwise_options', None)
    assert paramwise_options is None
    return obj_from_dict(optimizer_cfg, torch.optim,
                         dict(params=model.parameters())) 

    """
    obj_from_dict(info, parent=None, default_args=None)

    Args:
        info (dict): Object types and arguments.
        parent (:class:`module`): Module which may containing expected object
            classes.
        default_args (dict, optional): Default arguments for initializing the
            object.

    Returns:
      any type: Object built from the dict.
      return obj_type(**args) 
      e.g. SGD(lr=0.01, momentum=0.9, weight_decay=1e-4, ...)
    """


In [49]:
optimizer = build_optimizer(model, cfg.optimizer) #look at the func above


In [50]:
def batch_processor(model, data, train_mode):
    assert train_mode

    A_pred, loss = model(data, return_loss=True) #retrn_loss=true => output with loss 
    log_vars = OrderedDict()
    
    _, A, _, gtmat = data
    
    print('[shape] A_pred={} A={}'.format(A_pred.shape, A.shape))
    
    acc, p, r = online_evaluate(A_pred, A)
    
    log_vars['loss'] = loss.item()
    log_vars['accuracy'] = acc
    log_vars['precision'] = p
    log_vars['recall'] = r

    outputs = dict(loss=loss, log_vars=log_vars, num_samples=len(gtmat))

    return outputs

In [51]:
def _single_train(model, data_loaders, cfg): #called from train_gae
    if cfg.gpus > 1:
        raise NotImplemented
    # put model on gpus
    
    # model = MMDataParallel(model, device_ids=range(cfg.gpus)).cuda() 
    # build runner
    optimizer = build_optimizer(model, cfg.optimizer) #look at the func above

    runner = Runner(model, batch_processor, optimizer, cfg.work_dir,
                    cfg.log_level) #work_dir from --work_dir argument
    runner.register_training_hooks(cfg.lr_config, cfg.optimizer_config,
                                   cfg.checkpoint_config, cfg.log_config)#present related results

    if cfg.resume_from:
        runner.resume(cfg.resume_from)
    elif cfg.load_from:
        runner.load_checkpoint(cfg.load_from)
    runner.run(data_loaders, cfg.workflow, cfg.total_epochs)


# if cfg.resume_from:
#     runner.resume(cfg.resume_from)
# elif cfg.load_from:
#     runner.load_checkpoint(cfg.load_from)


In [38]:
if cfg.distributed:
    raise NotImplementedError
else:
    _single_train(model, data_loaders, cfg)

AssertionError: 